In [ ]:
!git clone https://github.com/ElanaPearl/interPLM.git
%cd interPLM
!pip install -e .
!pip install biopython

fatal: destination path 'interPLM' already exists and is not an empty directory.
/content/interPLM
Obtaining file:///content/interPLM
  Preparing metadata (setup.py) ... done
  Attempting uninstall: interplm
    Found existing installation: interplm 1.0.0
    Uninstalling interplm-1.0.0:
      Successfully uninstalled interplm-1.0.0
  Running setup.py develop for interplm


## Pobieranie danych

## Zbiór Danych: CATH Protein Domains


W bazie CATH istnieje ścisła hierarchia.

| Poziom | Oznaczenie | Nazwa | Liczba Klas* |
| --- | --- | --- | --- |
| **C** | `1` | **Class** | 5 |
| **A** | `1.10` | **Architecture** | 26 |
| **T** | `1.10.8` | **Topology** | 520  |
| **H** | `1.10.8.10` | **Homology** | 671 |

**Liczby klas są przybliżone i zależą od wersji bazy CATH.*

Będziemy trenować model na poziomie C.A.T, ponieważ niższe poziomy są zbyt ogólne, a poziom C.A.T.H jest zbyt rozdrobniony.


In [ ]:
%%bash
mkdir -p cath_data
cd cath_data

echo "Pobieranie etykiet..."
wget -q -nc ftp://orengoftp.biochem.ucl.ac.uk/cath/releases/latest-release/cath-classification-data/cath-domain-list.txt

echo "Pobieranie sekwencji..."
wget -q -nc ftp://orengoftp.biochem.ucl.ac.uk/cath/releases/latest-release/sequence-data/cath-domain-seqs.fa
ls -lh

Pobieranie etykiet...
Pobieranie sekwencji...
total 154M
-rw-r--r-- 1 szymon szymon  43M Jan 20 19:40 cath-domain-list.txt
-rw-r--r-- 1 szymon szymon 112M Jan 20 19:44 cath-domain-seqs.fa


In [3]:
!head -20 cath_data/cath-domain-list.txt

#---------------------------------------------------------------------
# FILE NAME:    CathDomainList.v4.4.0
# FILE DATE:    16.12.2024
#
# CATH VERSION: v4.4.0
# VERSION DATE: 16.12.2024
#
# FILE FORMAT:  Cath List File (CLF) Format 2.0
#
# FILE DESCRIPTION:
# Contains all classified protein domains in CATH
# for class 1 (mainly alpha), class 2 (mainly beta),
# class 3 (alpha and beta) and class 4 (few secondary structures).
#
# See 'README.file_formats' for file format information
#---------------------------------------------------------------------
1oaiA00     1    10     8    10     1     1     1     1     1    59 1.000
1go5A00     1    10     8    10     1     1     1     1     2    69 999.000
3frhA01     1    10     8    10     2     1     1     1     1    58 1.200
3friA01     1    10     8    10     2     1     1     1     2    54 1.800


In [ ]:
LABEL_FILE = 'cath_data/cath-domain-list.txt'
SEQ_FILE = 'cath_data/cath-domain-seqs.fa'
column_names = [
    'domain_id', 'class_C', 'arch_A', 'top_T', 'hom_H',
    's35', 's60', 's95', 's100', 's100_count', 'domain_len', 'resolution'
]

In [ ]:
df_labels = pd.read_csv(
    LABEL_FILE,
    sep=r'\s+',
    comment='#',
    header=None,
    names=column_names,
    usecols=['domain_id', 'class_C', 'arch_A', 'top_T', 'hom_H']
)

df_labels['target_label'] = (
    df_labels['class_C'].astype(str) + "." +
    df_labels['arch_A'].astype(str) + "." +
    df_labels['top_T'].astype(str)
)

print(df_labels[['class_C', 'arch_A', 'top_T', 'hom_H']].nunique())
print("Rozkład klas (C):")
print(df_labels['class_C'].value_counts())

top_families = df_labels['hom_H'].value_counts()
print("\nTop 5 najliczniejszych rodzin:")
print(top_families.head(5))

print("\n5 najmniej licznych rodzin:")
print(top_families.tail(5))

max_share = (top_families.iloc[0] / len(df_labels)) * 100
print(f"\nNajwiększa rodzina stanowi {max_share:.2f}% całego zbioru.")

In [ ]:
import pandas as pd
from Bio import SeqIO

df_labels = pd.read_csv(
    LABEL_FILE,
    sep=r'\s+',
    comment='#',
    header=None,
    names=column_names,
    usecols=['domain_id', 'class_C', 'arch_A', 'top_T', 'hom_H']
)
print(df_labels[['class_C', 'arch_A', 'top_T', 'hom_H']].nunique())
df_labels['target_label'] = (
    df_labels['class_C'].astype(str) + "." +
    df_labels['arch_A'].astype(str) + "." +
    df_labels['top_T'].astype(str)
)
df_labels.drop(columns=['class_C', 'arch_A', 'top_T', 'hom_H'], inplace=True, errors='ignore')


print(f"Wczytywanie sekwencji z {SEQ_FILE}")
seq_data = []

with open(SEQ_FILE, "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        original_id = record.id
        # original_id = cath|4_4_0|3avrA01/886-901_1246-1312_1380-1395
        try:
            part_with_id = original_id.split('|')[2]
            # part_with_id = 3avrA01/886-901_1246-1312_1380-1395

            clean_id = part_with_id.split('/')[0]
            # clean_id = 3avrA01
            seq_data.append({
                'domain_id': clean_id,
                'sequence': str(record.seq)
            })
        except IndexError:
            print(f"Pominięto nietypowy nagłówek: {original_id}")
            continue

df_seqs = pd.DataFrame(seq_data)


print("Łączenie sekwencji z etykietami")
full_dataset = pd.merge(df_labels, df_seqs, on='domain_id', how='inner')

print("-" * 50)
print(f"GOTOWY ZBIÓR DANYCH: {len(full_dataset)} próbek.")
print("-" * 50)
pd.set_option('display.max_colwidth', 50)
print(full_dataset.head())

Wczytywanie sekwencji z cath_data/cath-domain-seqs.fa...
Łączenie sekwencji z etykietami
--------------------------------------------------
GOTOWY ZBIÓR DANYCH: 601328 próbek.
--------------------------------------------------
  domain_id target_label                                           sequence
0   1oaiA00       1.10.8  PTLSPEQQEMLQAFSTQSGMNLEWSQKCLQDNNWDYTRSAQAFTHL...
1   1go5A00       1.10.8  PAPTPSSSPVPTLSPEQQEMLQAFSTQSGMNLEWSQKCLQDNNWDY...
2   3frhA01       1.10.8  YPMNINDALTSILASKKYRALCPDTVRRILTEEWGRHKSPKQTVEA...
3   3friA01       1.10.8  YPMNINDALTSILASKKYRALCPDTVRRILTEEWGRHKSPKQTVEA...
4   3b89A01       1.10.8  SLNINDALTSILASKKYRALCPDTVRRILTEEWGRHKSPKQTVEAA...


In [6]:
import torch
from transformers import AutoModel, AutoTokenizer
from interplm.sae.inference import load_sae_from_hf

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Używam urządzenia: {DEVICE}")

MODEL_NAME = "esm2-8m"
HF_MODEL_NAME = "facebook/esm2_t6_8M_UR50D"
LAYER_ID = 6

print("Ładowanie modelu ESM-2")
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_NAME)
base_model = AutoModel.from_pretrained(HF_MODEL_NAME).to(DEVICE)
base_model.eval()

print(f"Ładowanie SAE dla warstwy {LAYER_ID}...")
sae = load_sae_from_hf(plm_model=MODEL_NAME, plm_layer=LAYER_ID)
sae = sae.to(DEVICE)
sae.eval()


Używam urządzenia: cuda
Ładowanie modelu ESM-2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Ładowanie SAE dla warstwy 6...
Loading configs from /root/.cache/huggingface/hub/models--Elana--InterPLM-esm2-8m/snapshots/81d2429cd9dae7175f1dcd8b4c649a20cdc06c8c/layer_6/config.yaml
Loaded data type: <class 'interplm.train.configs.TrainingRunConfig'>
Data keys: Not a dict


ReLUSAE(
  (encoder): Linear(in_features=320, out_features=10240, bias=True)
  (decoder): Linear(in_features=10240, out_features=320, bias=False)
)

In [7]:
import numpy as np
from tqdm import tqdm

# --- OGRANICZENIE DANYCH DO TESTÓW ---
SAMPLE_SIZE = 100_000

if SAMPLE_SIZE:
    df_subset = full_dataset.sample(n=min(SAMPLE_SIZE, len(full_dataset)), random_state=42).copy()
else:
    df_subset = full_dataset.copy()

print(f"Przetwarzanie {len(df_subset)} próbek...")

def extract_sae_features(sequences, batch_size=32):
    all_features = []

    for i in tqdm(range(0, len(sequences), batch_size)):
        batch_seqs = sequences[i:i+batch_size]

        inputs = tokenizer(
            batch_seqs,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(DEVICE)

        with torch.no_grad():
            # Przepuszczamy sekwencje przez ESM-2
            outputs = base_model(**inputs, output_hidden_states=True)

            # Pobieramy hidden state z wybranej warstwy
            dense_acts = outputs.hidden_states[LAYER_ID]

            # Przepuszczamy pobraną warstwe InterPLM
            sae_acts = sae.encode(dense_acts)

            # SAE zwraca osobne cechy dla każdego aminokwasu
            # uśredniamy, aby uzyskać jeden wektor opisujący całe białko dla klasyfikatora.
            mask = inputs['attention_mask'].unsqueeze(-1).float()
            sum_features = torch.sum(sae_acts * mask, dim=1)
            count_tokens = torch.clamp(mask.sum(dim=1), min=1e-9)
            mean_features = sum_features / count_tokens

            all_features.append(mean_features.cpu().numpy())

    return np.vstack(all_features)

X = extract_sae_features(df_subset['sequence'].tolist())
y = df_subset['target_label'].values

print(f"\nWymiary macierzy cech X: {X.shape}")
print(f"Liczba etykiet y: {len(y)}")

Przetwarzanie 10000 próbek...


100%|██████████| 313/313 [00:56<00:00,  5.53it/s]



Wymiary macierzy cech X: (10000, 10240)
Liczba etykiet y: 10000


## Trening na CPU.
Działa ale dla dużej ilości danych zapycha RAM

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import numpy as np

# Filtrowanie rzadkich klas
unique_classes, counts = np.unique(y, return_counts=True)
valid_classes = unique_classes[counts > 1]

mask_valid = np.isin(y, valid_classes)
X_filtered = X[mask_valid]
y_filtered = y[mask_valid]

print(f"Oryginalna liczba próbek: {len(y)}")
print(f"Po usunięciu pojedynczych klas: {len(y_filtered)}")
print(f"Liczba unikalnych klas do przewidzenia: {len(valid_classes)}")


le = LabelEncoder()
y_encoded = le.fit_transform(y_filtered)

X_train, X_test, y_train, y_test = train_test_split(
    X_filtered,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

clf = LogisticRegression(
    max_iter=3000,
    solver='lbfgs',
    C=1.0,
    n_jobs=-1,
    verbose=1
)

print("\nTrening modelu")
clf.fit(X_train, y_train)

# Ewaluacja
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("-" * 40)
print(f"Accuracy: {acc:.2%}")
print("-" * 40)

# Wyświetlamy raport tylko dla klas obecnych w zbiorze testowym
unique_test_labels = np.unique(y_test)
target_names = le.inverse_transform(unique_test_labels)

print("\nRaport klasyfikacji:")
print(classification_report(
    y_test,
    y_pred,
    labels=unique_test_labels,
    target_names=target_names,
    zero_division=0
))

## Trening na GPU.


### Logistic Regression

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

unique_classes, counts = np.unique(y, return_counts=True)
valid_classes = unique_classes[counts > 1]
mask_valid = np.isin(y, valid_classes)
X_filtered = X[mask_valid]
y_filtered = y[mask_valid]

le = LabelEncoder()
y_encoded = le.fit_transform(y_filtered)

X_train, X_test, y_train, y_test = train_test_split(
    X_filtered, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Trenowanie na: {DEVICE}")

X_train_t = torch.tensor(X_train, dtype=torch.float32).to(DEVICE)
y_train_t = torch.tensor(y_train, dtype=torch.long).to(DEVICE)
X_test_t = torch.tensor(X_test, dtype=torch.float32).to(DEVICE)

input_dim = X_train.shape[1]
output_dim = len(valid_classes)

class LogisticRegressionPyTorch(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionPyTorch, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

model = LogisticRegressionPyTorch(input_dim, output_dim).to(DEVICE)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
EPOCHS = 1000

for epoch in range(EPOCHS):
    model.train()

    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0 or epoch == 0:
        print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}')


print("Ewaluacja")
model.eval()
with torch.no_grad():
    outputs = model(X_test_t)
    _, predicted = torch.max(outputs.data, 1)
    y_pred = predicted.cpu().numpy()


acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
p = precision_score(y_test, y_pred, average='macro', zero_division=0)
r = recall_score(y_test, y_pred, average='macro', zero_division=0)

print("-" * 30)
print(f"Accuracy:       {acc:.2%}")
print(f"F1:       {f1:.2%}")
print(f"Precision: {p:.2%}")
print(f"Recall:    {r:.2%}")
print("-" * 30)

unique_test_labels = np.unique(y_test)
target_names = le.inverse_transform(unique_test_labels)

print(classification_report(
    y_test,
    y_pred,
    labels=unique_test_labels,
    target_names=target_names,
    zero_division=0
))

Trenowanie na: cuda
Rozpoczynam trening na GPU...
Epoch [100/1000], Loss: 1.4051
Epoch [200/1000], Loss: 0.7287
Epoch [300/1000], Loss: 0.4689
Epoch [400/1000], Loss: 0.3354
Epoch [500/1000], Loss: 0.2556
Epoch [600/1000], Loss: 0.2030
Epoch [700/1000], Loss: 0.1659
Epoch [800/1000], Loss: 0.1384
Epoch [900/1000], Loss: 0.1174
Epoch [1000/1000], Loss: 0.1009
Ewaluacja...
----------------------------------------
Accuracy (GPU): 81.16%
----------------------------------------
              precision    recall  f1-score   support

     1.10.10       0.62      0.69      0.65        26
    1.10.100       1.00      1.00      1.00         1
   1.10.1030       0.00      0.00      0.00         1
   1.10.1040       0.40      0.67      0.50         3
   1.10.1050       1.00      1.00      1.00         1
   1.10.1060       1.00      1.00      1.00         1
   1.10.1070       1.00      1.00      1.00         1
    1.10.110       0.00      0.00      0.00         1
   1.10.1130       0.00      0.00 

### XGBoost

In [ ]:
import numpy as np
import xgboost as xgb
import torch
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=len(valid_classes),
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    device=DEVICE,
    random_state=42
)


print("Starting training")
model.fit(X_train, y_train)

print("Evaluation")
y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
p = precision_score(y_test, y_pred, average='macro', zero_division=0)
r = recall_score(y_test, y_pred, average='macro', zero_division=0)

print("-" * 30)
print(f"Accuracy:       {acc:.2%}")
print(f"F1:       {f1:.2%}")
print(f"Precision: {p:.2%}")
print(f"Recall:    {r:.2%}")
print("-" * 30)

unique_test_labels = np.unique(y_test)
target_names = le.inverse_transform(unique_test_labels)

print(classification_report(
    y_test,
    y_pred,
    labels=unique_test_labels,
    target_names=target_names,
    zero_division=0
))